In [1]:
# Import libraries
import numpy as np
import pandas
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import scanpy as sc
from datetime import datetime

# Global Variables
t1 = 0
patients = 0
genes = 0
k = 0

# Functions

# Define distance function which takes integer inputs which identify patient and centroid
def dist(patient_point, cluster_number):
    a = filtered_data[patient_point, :]
    b = centroids_array[cluster_number, :]
    dist = np.linalg.norm(a-b)
    return dist

def runtime_start():
    global t1 
    t1 = datetime.now().time()

def runtime_end():
    t2 = datetime.now().time()
    FMT = '%H:%M:%S.%f'
    elapsed = str(datetime.strptime(str(t2), FMT) - datetime.strptime(str(t1), FMT))
    return elapsed

# Import data
runtime_start()
data = sc.read_10x_mtx('./data/filtered_gene_bc_matrices/hg19/', var_names='gene_symbols', cache=True)

# Filter useless data
sc.pp.filter_genes(data, min_cells=1)
filtered_data = np.array(data._X.todense())

# Create Centroid Array by randomly picking 5 patients from data  
k = 5
patients = filtered_data.shape[0]
genes = filtered_data.shape[1]
centroids_numbers = np.random.randint(patients, size=k)
centroids_array = np.empty([0, genes])
i = 0


while i < k:
    randompatient = centroids_numbers[i]
    centroids_array = np.append(centroids_array, [filtered_data[randompatient, :]], axis = 0)
    i += 1

# Loop über alle Patienten
i = 0
nearest_centroid = np.zeros([patients, 1])    
while (i < patients):
    sml_distance = 0

    # While loop selecting every centroid
    j = 0
    while (j < k):

        if sml_distance == 0:
            sml_distance = dist(i,j)
            nearest_centroid[i, 0] = j
        elif dist(i,j) < sml_distance:
            sml_distance = dist(i,j)
            nearest_centroid[i, 0] = j
        else:
            pass
        j += 1
    i += 1

runtime_end()

'0:00:01.266386'

In [2]:
# Import libraries
import numpy as np
import pandas
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import scanpy as sc
from datetime import datetime

# Global Variables
t1 = 0
patients = 0
genes = 0
k = 0

# Functions

# Define distance function which takes integer inputs which identify patient and centroid
def dist(patient_point, cluster_number):
    a = filtered_data[patient_point, :]
    b = centroids_array[cluster_number, :]
    dist = np.linalg.norm(a-b)
    return dist

def runtime_start():
    global t1 
    t1 = datetime.now().time()

def runtime_end():
    t2 = datetime.now().time()
    FMT = '%H:%M:%S.%f'
    elapsed = str(datetime.strptime(str(t2), FMT) - datetime.strptime(str(t1), FMT))
    return elapsed

# Import data
runtime_start()
data = sc.read_10x_mtx('./data/filtered_gene_bc_matrices/hg19/', var_names='gene_symbols', cache=True)

# Filter useless data
sc.pp.filter_genes(data, min_cells=1)
filtered_data = np.array(data._X.todense())

# Create Centroid Array by randomly picking 5 patients from data  
k = 5
patients = filtered_data.shape[0]
genes = filtered_data.shape[1]
centroids_numbers = np.random.randint(patients, size=k)
centroids_array = np.empty([0, genes])
i = 0


while i < k:
    randompatient = centroids_numbers[i]
    centroids_array = np.append(centroids_array, [filtered_data[randompatient, :]], axis = 0)
    i += 1

# Loop über alle Patienten
i = 0
nearest_centroid = np.zeros([patients, 1])    
while (i < patients):
    sml_distance = 0

    # While loop selecting every centroid
    j = 0
    while (j < k):

        if sml_distance == 0:
            sml_distance = dist(i,j)
            nearest_centroid[i, 0] = j
        elif dist(i,j) < sml_distance:
            sml_distance = dist(i,j)
            nearest_centroid[i, 0] = j
        j += 1
    i += 1

runtime_end()

'0:00:01.261253'

In [3]:
# Import libraries
import numpy as np
import pandas
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import scanpy as sc
from datetime import datetime

# Global Variables
t1 = 0
patients = 0
genes = 0
k = 0

# Functions

# Define distance function which takes integer inputs which identify patient and centroid
def dist(patient_point, cluster_number):
    a = filtered_data[patient_point, :]
    b = centroids_array[cluster_number, :]
    dist = np.linalg.norm(a-b)
    return dist

def runtime_start():
    global t1 
    t1 = datetime.now().time()

def runtime_end():
    t2 = datetime.now().time()
    FMT = '%H:%M:%S.%f'
    elapsed = str(datetime.strptime(str(t2), FMT) - datetime.strptime(str(t1), FMT))
    return elapsed

# Import data
runtime_start()
data = sc.read_10x_mtx('./data/filtered_gene_bc_matrices/hg19/', var_names='gene_symbols', cache=True)

# Filter useless data
sc.pp.filter_genes(data, min_cells=1)
filtered_data = np.array(data._X.todense())

# Create Centroid Array by randomly picking 5 patients from data  
k = 5
patients = filtered_data.shape[0]
genes = filtered_data.shape[1]
centroids_numbers = np.random.randint(patients, size=k)
centroids_array = np.empty([0, genes])
i = 0


while i < k:
    randompatient = centroids_numbers[i]
    centroids_array = np.append(centroids_array, [filtered_data[randompatient, :]], axis = 0)
    i += 1

# Loop über alle Patienten
i = 0
nearest_centroid = np.zeros([patients, 1])    
while (i < patients):
    sml_distance = 0

    # While loop selecting every centroid
    j = 0
    while (j < k):

        if sml_distance == 0 or dist(i,j) < sml_distance:
            sml_distance = dist(i,j)
            nearest_centroid[i, 0] = j
        j += 1
    i += 1

runtime_end()

'0:00:01.233871'

In [4]:
# Import libraries
import numpy as np
import pandas
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import scanpy as sc
from datetime import datetime

# Global Variables
t1 = 0
patients = 0
genes = 0
k = 0

# Functions

# Define distance function which takes integer inputs which identify patient and centroid
def dist(patient_point, cluster_number):
    a = filtered_data[patient_point, :]
    b = centroids_array[cluster_number, :]
    dist = np.linalg.norm(a-b)
    return dist

def runtime_start():
    global t1 
    t1 = datetime.now().time()

def runtime_end():
    t2 = datetime.now().time()
    FMT = '%H:%M:%S.%f'
    elapsed = str(datetime.strptime(str(t2), FMT) - datetime.strptime(str(t1), FMT))
    return elapsed

# Import data
runtime_start()
data = sc.read_10x_mtx('./data/filtered_gene_bc_matrices/hg19/', var_names='gene_symbols', cache=True)

# Filter useless data
sc.pp.filter_genes(data, min_cells=1)
filtered_data = np.array(data._X.todense())

# Create Centroid Array by randomly picking 5 patients from data  
k = 5
patients = filtered_data.shape[0]
genes = filtered_data.shape[1]
centroids_numbers = np.random.randint(patients, size=k)
centroids_array = np.empty([0, genes])
i = 0


while i < k:
    randompatient = centroids_numbers[i]
    centroids_array = np.append(centroids_array, [filtered_data[randompatient, :]], axis = 0)
    i += 1

# Loop über alle Patienten
i = 0
nearest_centroid = np.zeros([patients, 1])    
while (i < patients):
    sml_distance = 0

    # While loop selecting every centroid
    j = 0
    while (j < k):

        if sml_distance == 0 or dist(i,j) < sml_distance:
            sml_distance = dist(i,j)
            nearest_centroid[i, 0] = j
        j += 1
    i += 1

runtime_end()

'0:00:01.192582'

In [5]:
# Import libraries
import numpy as np
import pandas
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import scanpy as sc
from datetime import datetime

# Global Variables
t1 = 0
patients = 0
genes = 0
k = 0

# Functions

# Define distance function which takes integer inputs which identify patient and centroid
def dist(patient_point, cluster_number):
    a = filtered_data[patient_point, :]
    b = centroids_array[cluster_number, :]
    dist = np.linalg.norm(a-b)
    return dist

def runtime_start():
    global t1 
    t1 = datetime.now().time()

def runtime_end():
    t2 = datetime.now().time()
    FMT = '%H:%M:%S.%f'
    elapsed = str(datetime.strptime(str(t2), FMT) - datetime.strptime(str(t1), FMT))
    return elapsed

# Import data
runtime_start()
data = sc.read_10x_mtx('./data/filtered_gene_bc_matrices/hg19/', var_names='gene_symbols', cache=True)

# Filter useless data
sc.pp.filter_genes(data, min_cells=1)
filtered_data = np.array(data._X.todense())

# Create Centroid Array by randomly picking 5 patients from data  
k = 5
patients = filtered_data.shape[0]
genes = filtered_data.shape[1]
centroids_numbers = np.random.randint(patients, size=k)
centroids_array = np.empty([0, genes])
i = 0


while i < k:
    randompatient = centroids_numbers[i]
    centroids_array = np.append(centroids_array, [filtered_data[randompatient, :]], axis = 0)
    i += 1

# Loop über alle Patienten
i = 0
nearest_centroid = np.zeros([patients, 1])    
while (i < patients):
    sml_distance = 0

    # While loop selecting every centroid
    j = 0
    while (j < k):

        if sml_distance == 0 or dist(i,j) < sml_distance:
            sml_distance = dist(i,j)
            nearest_centroid[i, 0] = j
        j += 1
    i += 1

runtime_end()

'0:00:01.298647'

In [6]:
# Import libraries
import numpy as np
import pandas
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import scanpy as sc
from datetime import datetime
from sklearn.decomposition import PCA

# Global Variables
t1 = 0
patients = 0
genes = 0
k = 0

# Functions

# Define distance function which takes integer inputs which identify patient and centroid
def dist(patient_point, cluster_number):
    a = filtered_data[patient_point, :]
    b = centroids_array[cluster_number, :]
    dist = np.linalg.norm(a-b)
    return dist

def runtime_start():
    global t1 
    t1 = datetime.now().time()

def runtime_end():
    t2 = datetime.now().time()
    FMT = '%H:%M:%S.%f'
    elapsed = str(datetime.strptime(str(t2), FMT) - datetime.strptime(str(t1), FMT))
    return elapsed

# Import data
runtime_start()
data = sc.read_10x_mtx('./data/filtered_gene_bc_matrices/hg19/', var_names='gene_symbols', cache=True)

# Filter useless data
sc.pp.filter_genes(data, min_cells=1)
filtered_data = np.array(data._X.todense())

# PCA
pca = PCA(n_components=2)
pca.fit(filtered_data)
print(pca.explained_variance_ratio_)
print(pca.singular_values_)

# Create Centroid Array by randomly picking 5 patients from data  
k = 5
patients = filtered_data.shape[0]
genes = filtered_data.shape[1]
centroids_numbers = np.random.randint(patients, size=k)
centroids_array = np.empty([0, genes])
i = 0


while i < k:
    randompatient = centroids_numbers[i]
    centroids_array = np.append(centroids_array, [filtered_data[randompatient, :]], axis = 0)
    i += 1

# Loop über alle Punkte
i = 0
nearest_centroid = np.zeros([patients, 1])    
while (i < patients):
    sml_distance = 0

    # While loop selecting every centroid
    j = 0
    while (j < k):

        if sml_distance == 0 or dist(i,j) < sml_distance:
            sml_distance = dist(i,j)
            nearest_centroid[i, 0] = j
        j += 1
    i += 1

runtime_end()

[0.26832464 0.21166636]
[3608.974  3205.3792]


'0:00:03.080834'

In [7]:
# Import libraries
import numpy as np
import pandas
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import scanpy as sc
from datetime import datetime
from sklearn.decomposition import PCA

# Global Variables
t1 = 0
patients = 0
genes = 0
k = 0

# Functions

# Define distance function which takes integer inputs which identify patient and centroid
def dist(patient_point, cluster_number):
    a = filtered_data[patient_point, :]
    b = centroids_array[cluster_number, :]
    dist = np.linalg.norm(a-b)
    return dist

def runtime_start():
    global t1 
    t1 = datetime.now().time()

def runtime_end():
    t2 = datetime.now().time()
    FMT = '%H:%M:%S.%f'
    elapsed = str(datetime.strptime(str(t2), FMT) - datetime.strptime(str(t1), FMT))
    return elapsed

# Import data
runtime_start()
data = sc.read_10x_mtx('./data/filtered_gene_bc_matrices/hg19/', var_names='gene_symbols', cache=True)

# Filter useless data
sc.pp.filter_genes(data, min_cells=1)
filtered_data = np.array(data._X.todense())

# PCA
pca = PCA(n_components=3)
pca.fit(filtered_data)
print(pca.explained_variance_ratio_)
print(pca.singular_values_)

# Create Centroid Array by randomly picking 5 patients from data  
k = 5
patients = filtered_data.shape[0]
genes = filtered_data.shape[1]
centroids_numbers = np.random.randint(patients, size=k)
centroids_array = np.empty([0, genes])
i = 0


while i < k:
    randompatient = centroids_numbers[i]
    centroids_array = np.append(centroids_array, [filtered_data[randompatient, :]], axis = 0)
    i += 1

# Loop über alle Punkte
i = 0
nearest_centroid = np.zeros([patients, 1])    
while (i < patients):
    sml_distance = 0

    # While loop selecting every centroid
    j = 0
    while (j < k):

        if sml_distance == 0 or dist(i,j) < sml_distance:
            sml_distance = dist(i,j)
            nearest_centroid[i, 0] = j
        j += 1
    i += 1

runtime_end()

[0.26832476 0.21166644 0.05710757]
[3608.9749 3205.3796 1664.9467]


'0:00:02.628660'

In [8]:
# Import libraries
import numpy as np
import pandas
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import scanpy as sc
from datetime import datetime
from sklearn.decomposition import PCA

# Global Variables
t1 = 0
patients = 0
genes = 0
k = 0

# Functions

# Define distance function which takes integer inputs which identify patient and centroid
def dist(patient_point, cluster_number):
    a = filtered_data[patient_point, :]
    b = centroids_array[cluster_number, :]
    dist = np.linalg.norm(a-b)
    return dist

def runtime_start():
    global t1 
    t1 = datetime.now().time()

def runtime_end():
    t2 = datetime.now().time()
    FMT = '%H:%M:%S.%f'
    elapsed = str(datetime.strptime(str(t2), FMT) - datetime.strptime(str(t1), FMT))
    return elapsed

# Import data
runtime_start()
data = sc.read_10x_mtx('./data/filtered_gene_bc_matrices/hg19/', var_names='gene_symbols', cache=True)

# Filter useless data
sc.pp.filter_genes(data, min_cells=1)
filtered_data = np.array(data._X.todense())

# PCA
pca = PCA(n_components=10)
pca.fit(filtered_data)
print(pca.explained_variance_ratio_)
print(pca.singular_values_)

# Create Centroid Array by randomly picking 5 patients from data  
k = 5
patients = filtered_data.shape[0]
genes = filtered_data.shape[1]
centroids_numbers = np.random.randint(patients, size=k)
centroids_array = np.empty([0, genes])
i = 0


while i < k:
    randompatient = centroids_numbers[i]
    centroids_array = np.append(centroids_array, [filtered_data[randompatient, :]], axis = 0)
    i += 1

# Loop über alle Punkte
i = 0
nearest_centroid = np.zeros([patients, 1])    
while (i < patients):
    sml_distance = 0

    # While loop selecting every centroid
    j = 0
    while (j < k):

        if sml_distance == 0 or dist(i,j) < sml_distance:
            sml_distance = dist(i,j)
            nearest_centroid[i, 0] = j
        j += 1
    i += 1

runtime_end()

[0.26832503 0.21166629 0.05710753 0.03913825 0.02778206 0.02526641
 0.01696016 0.01315827 0.01083701 0.00696942]
[3608.9766  3205.3787  1664.9462  1378.3331  1161.2766  1107.4529
  907.3366   799.1953   725.28406  581.63654]


'0:00:02.849417'

In [9]:
# Import libraries
import numpy as np
import pandas
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import scanpy as sc
from datetime import datetime
from sklearn.decomposition import PCA

# Global Variables
t1 = 0
patients = 0
genes = 0
k = 0

# Functions

# Define distance function which takes integer inputs which identify patient and centroid
def dist(patient_point, cluster_number):
    a = filtered_data[patient_point, :]
    b = centroids_array[cluster_number, :]
    dist = np.linalg.norm(a-b)
    return dist

def runtime_start():
    global t1 
    t1 = datetime.now().time()

def runtime_end():
    t2 = datetime.now().time()
    FMT = '%H:%M:%S.%f'
    elapsed = str(datetime.strptime(str(t2), FMT) - datetime.strptime(str(t1), FMT))
    return elapsed

# Import data
runtime_start()
data = sc.read_10x_mtx('./data/filtered_gene_bc_matrices/hg19/', var_names='gene_symbols', cache=True)

# Filter useless data
sc.pp.filter_genes(data, min_cells=1)
filtered_data = np.array(data._X.todense())

# PCA
pca = PCA(n_components == 'mle')
pca.fit(filtered_data)
print(pca.explained_variance_ratio_)
print(pca.singular_values_)

# Create Centroid Array by randomly picking 5 patients from data  
k = 5
patients = filtered_data.shape[0]
genes = filtered_data.shape[1]
centroids_numbers = np.random.randint(patients, size=k)
centroids_array = np.empty([0, genes])
i = 0


while i < k:
    randompatient = centroids_numbers[i]
    centroids_array = np.append(centroids_array, [filtered_data[randompatient, :]], axis = 0)
    i += 1

# Loop über alle Punkte
i = 0
nearest_centroid = np.zeros([patients, 1])    
while (i < patients):
    sml_distance = 0

    # While loop selecting every centroid
    j = 0
    while (j < k):

        if sml_distance == 0 or dist(i,j) < sml_distance:
            sml_distance = dist(i,j)
            nearest_centroid[i, 0] = j
        j += 1
    i += 1

runtime_end()

NameError: name 'n_components' is not defined

In [10]:
# Import libraries
import numpy as np
import pandas
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import scanpy as sc
from datetime import datetime
from sklearn.decomposition import PCA

# Global Variables
t1 = 0
patients = 0
genes = 0
k = 0

# Functions

# Define distance function which takes integer inputs which identify patient and centroid
def dist(patient_point, cluster_number):
    a = filtered_data[patient_point, :]
    b = centroids_array[cluster_number, :]
    dist = np.linalg.norm(a-b)
    return dist

def runtime_start():
    global t1 
    t1 = datetime.now().time()

def runtime_end():
    t2 = datetime.now().time()
    FMT = '%H:%M:%S.%f'
    elapsed = str(datetime.strptime(str(t2), FMT) - datetime.strptime(str(t1), FMT))
    return elapsed

# Import data
runtime_start()
data = sc.read_10x_mtx('./data/filtered_gene_bc_matrices/hg19/', var_names='gene_symbols', cache=True)

# Filter useless data
sc.pp.filter_genes(data, min_cells=1)
filtered_data = np.array(data._X.todense())

# PCA
pca = PCA(n_components == 'mle', svd_solver == 'full')
pca.fit(filtered_data)
print(pca.explained_variance_ratio_)
print(pca.singular_values_)

# Create Centroid Array by randomly picking 5 patients from data  
k = 5
patients = filtered_data.shape[0]
genes = filtered_data.shape[1]
centroids_numbers = np.random.randint(patients, size=k)
centroids_array = np.empty([0, genes])
i = 0


while i < k:
    randompatient = centroids_numbers[i]
    centroids_array = np.append(centroids_array, [filtered_data[randompatient, :]], axis = 0)
    i += 1

# Loop über alle Punkte
i = 0
nearest_centroid = np.zeros([patients, 1])    
while (i < patients):
    sml_distance = 0

    # While loop selecting every centroid
    j = 0
    while (j < k):

        if sml_distance == 0 or dist(i,j) < sml_distance:
            sml_distance = dist(i,j)
            nearest_centroid[i, 0] = j
        j += 1
    i += 1

runtime_end()

NameError: name 'n_components' is not defined

In [11]:
# Import libraries
import numpy as np
import pandas
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import scanpy as sc
from datetime import datetime
from sklearn.decomposition import PCA

# Global Variables
t1 = 0
patients = 0
genes = 0
k = 0

# Functions

# Define distance function which takes integer inputs which identify patient and centroid
def dist(patient_point, cluster_number):
    a = filtered_data[patient_point, :]
    b = centroids_array[cluster_number, :]
    dist = np.linalg.norm(a-b)
    return dist

def runtime_start():
    global t1 
    t1 = datetime.now().time()

def runtime_end():
    t2 = datetime.now().time()
    FMT = '%H:%M:%S.%f'
    elapsed = str(datetime.strptime(str(t2), FMT) - datetime.strptime(str(t1), FMT))
    return elapsed

# Import data
runtime_start()
data = sc.read_10x_mtx('./data/filtered_gene_bc_matrices/hg19/', var_names='gene_symbols', cache=True)

# Filter useless data
sc.pp.filter_genes(data, min_cells=1)
filtered_data = np.array(data._X.todense())

# PCA
pca = PCA(n_components=2)
pca.fit(filtered_data)
print(pca.explained_variance_ratio_)
print(pca.singular_values_)

# Create Centroid Array by randomly picking 5 patients from data  
k = 5
patients = filtered_data.shape[0]
genes = filtered_data.shape[1]
centroids_numbers = np.random.randint(patients, size=k)
centroids_array = np.empty([0, genes])
i = 0


while i < k:
    randompatient = centroids_numbers[i]
    centroids_array = np.append(centroids_array, [filtered_data[randompatient, :]], axis = 0)
    i += 1

# Loop über alle Punkte
i = 0
nearest_centroid = np.zeros([patients, 1])    
while (i < patients):
    sml_distance = 0

    # While loop selecting every centroid
    j = 0
    while (j < k):

        if sml_distance == 0 or dist(i,j) < sml_distance:
            sml_distance = dist(i,j)
            nearest_centroid[i, 0] = j
        j += 1
    i += 1

runtime_end()

[0.2683247  0.21166652]
[3608.9744 3205.3804]


'0:00:02.790153'

In [12]:
# Import libraries
import numpy as np
import pandas
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import scanpy as sc
from datetime import datetime
from sklearn.decomposition import PCA

# Global Variables
t1 = 0
patients = 0
genes = 0
k = 0

# Functions

# Define distance function which takes integer inputs which identify patient and centroid
def dist(patient_point, cluster_number):
    a = filtered_data[patient_point, :]
    b = centroids_array[cluster_number, :]
    dist = np.linalg.norm(a-b)
    return dist

def runtime_start():
    global t1 
    t1 = datetime.now().time()

def runtime_end():
    t2 = datetime.now().time()
    FMT = '%H:%M:%S.%f'
    elapsed = str(datetime.strptime(str(t2), FMT) - datetime.strptime(str(t1), FMT))
    return elapsed

# Import data
runtime_start()
data = sc.read_10x_mtx('./data/filtered_gene_bc_matrices/hg19/', var_names='gene_symbols', cache=True)

# Filter useless data
sc.pp.filter_genes(data, min_cells=1)
filtered_data = np.array(data._X.todense())

# PCA
pca = PCA(n_components=3)
pca.fit(filtered_data)
print(pca.explained_variance_ratio_)
print(pca.singular_values_)

# Create Centroid Array by randomly picking 5 patients from data  
k = 5
patients = filtered_data.shape[0]
genes = filtered_data.shape[1]
centroids_numbers = np.random.randint(patients, size=k)
centroids_array = np.empty([0, genes])
i = 0


while i < k:
    randompatient = centroids_numbers[i]
    centroids_array = np.append(centroids_array, [filtered_data[randompatient, :]], axis = 0)
    i += 1

# Loop über alle Punkte
i = 0
nearest_centroid = np.zeros([patients, 1])    
while (i < patients):
    sml_distance = 0

    # While loop selecting every centroid
    j = 0
    while (j < k):

        if sml_distance == 0 or dist(i,j) < sml_distance:
            sml_distance = dist(i,j)
            nearest_centroid[i, 0] = j
        j += 1
    i += 1

runtime_end()

[0.26832476 0.21166629 0.05710755]
[3608.9749 3205.3787 1664.9463]


'0:00:02.432816'

In [13]:
# Import libraries
import numpy as np
import pandas
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import scanpy as sc
from datetime import datetime
from sklearn.decomposition import PCA

# Global Variables
t1 = 0
patients = 0
genes = 0
k = 0

# Functions

# Define distance function which takes integer inputs which identify patient and centroid
def dist(patient_point, cluster_number):
    a = filtered_data[patient_point, :]
    b = centroids_array[cluster_number, :]
    dist = np.linalg.norm(a-b)
    return dist

def runtime_start():
    global t1 
    t1 = datetime.now().time()

def runtime_end():
    t2 = datetime.now().time()
    FMT = '%H:%M:%S.%f'
    elapsed = str(datetime.strptime(str(t2), FMT) - datetime.strptime(str(t1), FMT))
    return elapsed

# Import data
runtime_start()
data = sc.read_10x_mtx('./data/filtered_gene_bc_matrices/hg19/', var_names='gene_symbols', cache=True)

# Filter useless data
sc.pp.filter_genes(data, min_cells=1)
filtered_data = np.array(data._X.todense())

# PCA
pca = PCA(n_components=3)
pca.fit(filtered_data)
print(sum(pca.explained_variance_ratio_)
print(pca.singular_values_)

# Create Centroid Array by randomly picking 5 patients from data  
k = 5
patients = filtered_data.shape[0]
genes = filtered_data.shape[1]
centroids_numbers = np.random.randint(patients, size=k)
centroids_array = np.empty([0, genes])
i = 0


while i < k:
    randompatient = centroids_numbers[i]
    centroids_array = np.append(centroids_array, [filtered_data[randompatient, :]], axis = 0)
    i += 1

# Loop über alle Punkte
i = 0
nearest_centroid = np.zeros([patients, 1])    
while (i < patients):
    sml_distance = 0

    # While loop selecting every centroid
    j = 0
    while (j < k):

        if sml_distance == 0 or dist(i,j) < sml_distance:
            sml_distance = dist(i,j)
            nearest_centroid[i, 0] = j
        j += 1
    i += 1

runtime_end()

SyntaxError: invalid syntax (<ipython-input-13-99c861cbfd0a>, line 47)

In [14]:
# Import libraries
import numpy as np
import pandas
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import scanpy as sc
from datetime import datetime
from sklearn.decomposition import PCA

# Global Variables
t1 = 0
patients = 0
genes = 0
k = 0

# Functions

# Define distance function which takes integer inputs which identify patient and centroid
def dist(patient_point, cluster_number):
    a = filtered_data[patient_point, :]
    b = centroids_array[cluster_number, :]
    dist = np.linalg.norm(a-b)
    return dist

def runtime_start():
    global t1 
    t1 = datetime.now().time()

def runtime_end():
    t2 = datetime.now().time()
    FMT = '%H:%M:%S.%f'
    elapsed = str(datetime.strptime(str(t2), FMT) - datetime.strptime(str(t1), FMT))
    return elapsed

# Import data
runtime_start()
data = sc.read_10x_mtx('./data/filtered_gene_bc_matrices/hg19/', var_names='gene_symbols', cache=True)

# Filter useless data
sc.pp.filter_genes(data, min_cells=1)
filtered_data = np.array(data._X.todense())

# PCA
pca = PCA(n_components=3)
pca_data = pca.fit_transform(filtered_data)
print(pca.explained_variance_ratio_)
print(pca.singular_values_)

# Create Centroid Array by randomly picking 5 patients from data  
k = 5
patients = filtered_data.shape[0]
genes = filtered_data.shape[1]
centroids_numbers = np.random.randint(patients, size=k)
centroids_array = np.empty([0, genes])
i = 0


while i < k:
    randompatient = centroids_numbers[i]
    centroids_array = np.append(centroids_array, [filtered_data[randompatient, :]], axis = 0)
    i += 1

# Loop über alle Punkte
i = 0
nearest_centroid = np.zeros([patients, 1])    
while (i < patients):
    sml_distance = 0

    # While loop selecting every centroid
    j = 0
    while (j < k):

        if sml_distance == 0 or dist(i,j) < sml_distance:
            sml_distance = dist(i,j)
            nearest_centroid[i, 0] = j
        j += 1
    i += 1

runtime_end()

[0.26832476 0.2116664  0.05710753]
[3608.9749 3205.3794 1664.9462]


'0:00:02.936306'

In [15]:
np.random.randint(0,1)


0

In [16]:
# Import libraries
import numpy as np
import pandas
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import scanpy as sc
from datetime import datetime
from sklearn.decomposition import PCA

# Global Variables
t1 = 0
patients = 0
genes = 0
k = 0

# Functions

# Define distance function which takes integer inputs which identify patient and centroid
def dist(patient_point, cluster_number):
    a = filtered_data[patient_point, :]
    b = centroids_array[cluster_number, :]
    dist = np.linalg.norm(a-b)
    return dist

def runtime_start():
    global t1 
    t1 = datetime.now().time()

def runtime_end():
    t2 = datetime.now().time()
    FMT = '%H:%M:%S.%f'
    elapsed = str(datetime.strptime(str(t2), FMT) - datetime.strptime(str(t1), FMT))
    return elapsed

# Import data
runtime_start()
data = sc.read_10x_mtx('./data/filtered_gene_bc_matrices/hg19/', var_names='gene_symbols', cache=True)

# Filter useless data
sc.pp.filter_genes(data, min_cells=1)
filtered_data = np.array(data._X.todense())

# PCA
pca = PCA(n_components=2)
pca_data = pca.fit_transform(filtered_data)
print(pca.explained_variance_ratio_)
print(pca.singular_values_)

# Create Centroid Array by randomly picking 5 patients from data  
k = 5
patients = filtered_data.shape[0]
genes = filtered_data.shape[1]
centroids_numbers = np.random.randint(patients, size=k)
centroids_array = np.empty([0, genes])
i = 0


while i < k:
    randompatient = centroids_numbers[i]
    centroids_array = np.append(centroids_array, [filtered_data[randompatient, :]], axis = 0)
    i += 1

# Loop über alle Punkte
i = 0
nearest_centroid = np.zeros([patients, 1])    
while (i < patients):
    sml_distance = 0

    # While loop selecting every centroid
    j = 0
    while (j < k):

        if sml_distance == 0 or dist(i,j) < sml_distance:
            sml_distance = dist(i,j)
            nearest_centroid[i, 0] = j
        j += 1
    i += 1

runtime_end()

[0.2683248 0.2116664]
[3608.975  3205.3794]


'0:00:02.432873'

In [17]:
# Import libraries
import numpy as np
import pandas
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import scanpy as sc
from datetime import datetime
from sklearn.decomposition import PCA

# Global Variables
t1 = 0
patients = 0
genes = 0
k = 0

# Functions

# Define distance function which takes integer inputs which identify patient and centroid
def dist(patient_point, cluster_number):
    a = filtered_data[patient_point, :]
    b = centroids_array[cluster_number, :]
    dist = np.linalg.norm(a-b)
    return dist

def runtime_start():
    global t1 
    t1 = datetime.now().time()

def runtime_end():
    t2 = datetime.now().time()
    FMT = '%H:%M:%S.%f'
    elapsed = str(datetime.strptime(str(t2), FMT) - datetime.strptime(str(t1), FMT))
    return elapsed

# Import data
runtime_start()
data = sc.read_10x_mtx('./data/filtered_gene_bc_matrices/hg19/', var_names='gene_symbols', cache=True)

# Filter useless data
sc.pp.filter_genes(data, min_cells=1)
filtered_data = np.array(data._X.todense())

# PCA
pca = PCA(n_components=2)
pca_data = pca.fit_transform(filtered_data)
print(sum(pca.explained_variance_ratio_)
print(pca.singular_values_)

# Create Centroid Array by randomly picking 5 patients from data  
k = 5
patients = filtered_data.shape[0]
genes = filtered_data.shape[1]
centroids_numbers = np.random.randint(patients, size=k)
centroids_array = np.empty([0, genes])
i = 0


while i < k:
    randompatient = centroids_numbers[i]
    centroids_array = np.append(centroids_array, [filtered_data[randompatient, :]], axis = 0)
    i += 1

# Loop über alle Punkte
i = 0
nearest_centroid = np.zeros([patients, 1])    
while (i < patients):
    sml_distance = 0

    # While loop selecting every centroid
    j = 0
    while (j < k):

        if sml_distance == 0 or dist(i,j) < sml_distance:
            sml_distance = dist(i,j)
            nearest_centroid[i, 0] = j
        j += 1
    i += 1

runtime_end()

SyntaxError: invalid syntax (<ipython-input-17-3b724659dd7a>, line 47)

In [18]:
# Import libraries
import numpy as np
import pandas
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import scanpy as sc
from datetime import datetime
from sklearn.decomposition import PCA

# Global Variables
t1 = 0
patients = 0
genes = 0
k = 0

# Functions

# Define distance function which takes integer inputs which identify patient and centroid
def dist(patient_point, cluster_number):
    a = filtered_data[patient_point, :]
    b = centroids_array[cluster_number, :]
    dist = np.linalg.norm(a-b)
    return dist

def runtime_start():
    global t1 
    t1 = datetime.now().time()

def runtime_end():
    t2 = datetime.now().time()
    FMT = '%H:%M:%S.%f'
    elapsed = str(datetime.strptime(str(t2), FMT) - datetime.strptime(str(t1), FMT))
    return elapsed

# Import data
runtime_start()
data = sc.read_10x_mtx('./data/filtered_gene_bc_matrices/hg19/', var_names='gene_symbols', cache=True)

# Filter useless data
sc.pp.filter_genes(data, min_cells=1)
filtered_data = np.array(data._X.todense())

# PCA
pca = PCA(n_components=2)
pca_data = pca.fit_transform(filtered_data)
print(sum(pca.explained_variance_ratio_))
print(pca.singular_values_)

# Create Centroid Array by randomly picking 5 patients from data  
k = 5
patients = filtered_data.shape[0]
genes = filtered_data.shape[1]
centroids_numbers = np.random.randint(patients, size=k)
centroids_array = np.empty([0, genes])
i = 0


while i < k:
    randompatient = centroids_numbers[i]
    centroids_array = np.append(centroids_array, [filtered_data[randompatient, :]], axis = 0)
    i += 1

# Loop über alle Punkte
i = 0
nearest_centroid = np.zeros([patients, 1])    
while (i < patients):
    sml_distance = 0

    # While loop selecting every centroid
    j = 0
    while (j < k):

        if sml_distance == 0 or dist(i,j) < sml_distance:
            sml_distance = dist(i,j)
            nearest_centroid[i, 0] = j
        j += 1
    i += 1

runtime_end()

0.4799911826848984
[3608.9746 3205.38  ]


'0:00:02.713270'

In [19]:
# Import libraries
import numpy as np
import pandas
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import scanpy as sc
from datetime import datetime
from sklearn.decomposition import PCA

# Global Variables
t1 = 0
patients = 0
genes = 0
k = 0

# Functions

# Define distance function which takes integer inputs which identify patient and centroid
def dist(patient_point, cluster_number):
    a = filtered_data[patient_point, :]
    b = centroids_array[cluster_number, :]
    dist = np.linalg.norm(a-b)
    return dist

def runtime_start():
    global t1 
    t1 = datetime.now().time()

def runtime_end():
    t2 = datetime.now().time()
    FMT = '%H:%M:%S.%f'
    elapsed = str(datetime.strptime(str(t2), FMT) - datetime.strptime(str(t1), FMT))
    return elapsed

# Import data
runtime_start()
data = sc.read_10x_mtx('./data/filtered_gene_bc_matrices/hg19/', var_names='gene_symbols', cache=True)

# Filter useless data
sc.pp.filter_genes(data, min_cells=1)
filtered_data = np.array(data._X.todense())

# PCA
pca = PCA(n_components=10)
pca_data = pca.fit_transform(filtered_data)
print(sum(pca.explained_variance_ratio_))
print(pca.singular_values_)

# Create Centroid Array by randomly picking 5 patients from data  
k = 5
patients = filtered_data.shape[0]
genes = filtered_data.shape[1]
centroids_numbers = np.random.randint(patients, size=k)
centroids_array = np.empty([0, genes])
i = 0


while i < k:
    randompatient = centroids_numbers[i]
    centroids_array = np.append(centroids_array, [filtered_data[randompatient, :]], axis = 0)
    i += 1

# Loop über alle Punkte
i = 0
nearest_centroid = np.zeros([patients, 1])    
while (i < patients):
    sml_distance = 0

    # While loop selecting every centroid
    j = 0
    while (j < k):

        if sml_distance == 0 or dist(i,j) < sml_distance:
            sml_distance = dist(i,j)
            nearest_centroid[i, 0] = j
        j += 1
    i += 1

runtime_end()

0.6772103197872639
[3608.9756 3205.379  1664.9464 1378.3333 1161.2766 1107.4525  907.3374
  799.1957  725.2841  581.6353]


'0:00:03.065216'

In [20]:
# Import libraries
import numpy as np
import pandas
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import scanpy as sc
from datetime import datetime
from sklearn.decomposition import PCA

# Global Variables
t1 = 0
patients = 0
genes = 0
k = 0

# Functions

# Define distance function which takes integer inputs which identify patient and centroid
def dist(patient_point, cluster_number):
    a = filtered_data[patient_point, :]
    b = centroids_array[cluster_number, :]
    dist = np.linalg.norm(a-b)
    return dist

def runtime_start():
    global t1 
    t1 = datetime.now().time()

def runtime_end():
    t2 = datetime.now().time()
    FMT = '%H:%M:%S.%f'
    elapsed = str(datetime.strptime(str(t2), FMT) - datetime.strptime(str(t1), FMT))
    return elapsed

# Import data
runtime_start()
data = sc.read_10x_mtx('./data/filtered_gene_bc_matrices/hg19/', var_names='gene_symbols', cache=True)

# Filter useless data
sc.pp.filter_genes(data, min_cells=3)
filtered_data = np.array(data._X.todense())

# PCA
pca = PCA(n_components=2)
pca_data = pca.fit_transform(filtered_data)
print(sum(pca.explained_variance_ratio_))
print(pca.singular_values_)

# Create Centroid Array by randomly picking 5 patients from data  
k = 5
patients = filtered_data.shape[0]
genes = filtered_data.shape[1]
centroids_numbers = np.random.randint(patients, size=k)
centroids_array = np.empty([0, genes])
i = 0


while i < k:
    randompatient = centroids_numbers[i]
    centroids_array = np.append(centroids_array, [filtered_data[randompatient, :]], axis = 0)
    i += 1

# Loop über alle Punkte
i = 0
nearest_centroid = np.zeros([patients, 1])    
while (i < patients):
    sml_distance = 0

    # While loop selecting every centroid
    j = 0
    while (j < k):

        if sml_distance == 0 or dist(i,j) < sml_distance:
            sml_distance = dist(i,j)
            nearest_centroid[i, 0] = j
        j += 1
    i += 1

runtime_end()

0.48006805777549744
[3608.9736 3205.3787]


'0:00:02.277360'

In [21]:
# Import libraries
import numpy as np
import pandas
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import scanpy as sc
from datetime import datetime
from sklearn.decomposition import PCA

# Global Variables
t1 = 0
patients = 0
genes = 0
k = 0

# Functions

# Define distance function which takes integer inputs which identify patient and centroid
def dist(patient_point, cluster_number):
    a = filtered_data[patient_point, :]
    b = centroids_array[cluster_number, :]
    dist = np.linalg.norm(a-b)
    return dist

def runtime_start():
    global t1 
    t1 = datetime.now().time()

def runtime_end():
    t2 = datetime.now().time()
    FMT = '%H:%M:%S.%f'
    elapsed = str(datetime.strptime(str(t2), FMT) - datetime.strptime(str(t1), FMT))
    return elapsed

# Import data
runtime_start()
data = sc.read_10x_mtx('./data/filtered_gene_bc_matrices/hg19/', var_names='gene_symbols', cache=True)

# Filter useless data
sc.pp.filter_genes(data, min_cells=10)
filtered_data = np.array(data._X.todense())

# PCA
pca = PCA(n_components=2)
pca_data = pca.fit_transform(filtered_data)
print(sum(pca.explained_variance_ratio_))
print(pca.singular_values_)

# Create Centroid Array by randomly picking 5 patients from data  
k = 5
patients = filtered_data.shape[0]
genes = filtered_data.shape[1]
centroids_numbers = np.random.randint(patients, size=k)
centroids_array = np.empty([0, genes])
i = 0


while i < k:
    randompatient = centroids_numbers[i]
    centroids_array = np.append(centroids_array, [filtered_data[randompatient, :]], axis = 0)
    i += 1

# Loop über alle Punkte
i = 0
nearest_centroid = np.zeros([patients, 1])    
while (i < patients):
    sml_distance = 0

    # While loop selecting every centroid
    j = 0
    while (j < k):

        if sml_distance == 0 or dist(i,j) < sml_distance:
            sml_distance = dist(i,j)
            nearest_centroid[i, 0] = j
        j += 1
    i += 1

runtime_end()

0.4804442226886749
[3608.9666 3205.3699]


'0:00:02.246138'

In [22]:
# Import libraries
import numpy as np
import pandas
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import scanpy as sc
from datetime import datetime
from sklearn.decomposition import PCA

# Global Variables
t1 = 0
patients = 0
genes = 0
k = 0

# Functions

# Define distance function which takes integer inputs which identify patient and centroid
def dist(patient_point, cluster_number):
    a = filtered_data[patient_point, :]
    b = centroids_array[cluster_number, :]
    dist = np.linalg.norm(a-b)
    return dist

def runtime_start():
    global t1 
    t1 = datetime.now().time()

def runtime_end():
    t2 = datetime.now().time()
    FMT = '%H:%M:%S.%f'
    elapsed = str(datetime.strptime(str(t2), FMT) - datetime.strptime(str(t1), FMT))
    return elapsed

# Import data
runtime_start()
data = sc.read_10x_mtx('./data/filtered_gene_bc_matrices/hg19/', var_names='gene_symbols', cache=True)

# Filter useless data
sc.pp.filter_genes(data, min_cells=1)
filtered_data = np.array(data._X.todense())

# PCA
pca = PCA(n_components=2)
pca_data = pca.fit_transform(filtered_data)
print(sum(pca.explained_variance_ratio_))
print(pca.singular_values_)

# Create Centroid Array by randomly picking 5 patients from data  
k = 5
patients = filtered_data.shape[0]
genes = filtered_data.shape[1]
centroids_numbers = np.random.randint(patients, size=k)
centroids_array = np.empty([0, genes])
i = 0


while i < k:
    randompatient = centroids_numbers[i]
    centroids_array = np.append(centroids_array, [filtered_data[randompatient, :]], axis = 0)
    i += 1

# Loop über alle Punkte
i = 0
nearest_centroid = np.zeros([patients, 1])    
while (i < patients):
    sml_distance = 0

    # While loop selecting every centroid
    j = 0
    while (j < k):

        if sml_distance == 0 or dist(i,j) < sml_distance:
            sml_distance = dist(i,j)
            nearest_centroid[i, 0] = j
        j += 1
    i += 1

runtime_end()

0.47999121248722076
[3608.9749 3205.38  ]


'0:00:02.727281'

In [23]:
# Import libraries
import numpy as np
import pandas
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import scanpy as sc
from datetime import datetime
from sklearn.decomposition import PCA

# Global Variables
t1 = 0
patients = 0
genes = 0
k = 0

# Functions

# Define distance function which takes integer inputs which identify patient and centroid
def dist(patient_point, cluster_number):
    a = pca_data[patient_point, :]
    b = centroids_array[cluster_number, :]
    dist = np.linalg.norm(a-b)
    return dist

def runtime_start():
    global t1 
    t1 = datetime.now().time()

def runtime_end():
    t2 = datetime.now().time()
    FMT = '%H:%M:%S.%f'
    elapsed = str(datetime.strptime(str(t2), FMT) - datetime.strptime(str(t1), FMT))
    return elapsed

# Import data
runtime_start()
data = sc.read_10x_mtx('./data/filtered_gene_bc_matrices/hg19/', var_names='gene_symbols', cache=True)

# Filter useless data
sc.pp.filter_genes(data, min_cells=1)
filtered_data = np.array(data._X.todense())

# PCA
pca = PCA(n_components=2)
pca_data = pca.fit_transform(filtered_data)
print(sum(pca.explained_variance_ratio_))
print(pca.singular_values_)

def random_patients():
    # Create Centroid Array by randomly picking 5 patients from data  
    k = 5
    patients = pca_data.shape[0]
    genes = pca_data.shape[1]
    centroids_numbers = np.random.randint(patients, size=k)
    centroids_array = np.empty([0, genes])
    i = 0

def random_start_centroids(k):
    # Pick random start sample 
    while i < k:
        randompatient = centroids_numbers[i]
        centroids_array = np.append(centroids_array, [pca_data[randompatient, :]], axis = 0)
        i += 1

def assign_centroids():
    # Assign closest Centroid
    # Loop über alle Punkte
    i = 0
    nearest_centroid = np.zeros([patients, 1])    
    while (i < patients):
        sml_distance = 0

        # While loop selecting every centroid
        j = 0
        while (j < k):

            if sml_distance == 0 or dist(i,j) < sml_distance:
                sml_distance = dist(i,j)
                nearest_centroid[i, 0] = j
            j += 1
        i += 1

def kmeans():
    random_patients()
    random_start_centroids()
    assign_centroids()

runtime_end()

0.4799913763999939
[3608.9753 3205.3804]


'0:00:01.561114'

In [24]:
# Import libraries
import numpy as np
import pandas
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import scanpy as sc
from datetime import datetime
from sklearn.decomposition import PCA

# Global Variables
t1 = 0
patients = 0
genes = 0
k = 0

# Functions

# Define distance function which takes integer inputs which identify patient and centroid
def dist(patient_point, cluster_number):
    a = pca_data[patient_point, :]
    b = centroids_array[cluster_number, :]
    dist = np.linalg.norm(a-b)
    return dist

def runtime_start():
    global t1 
    t1 = datetime.now().time()

def runtime_end():
    t2 = datetime.now().time()
    FMT = '%H:%M:%S.%f'
    elapsed = str(datetime.strptime(str(t2), FMT) - datetime.strptime(str(t1), FMT))
    return elapsed

# Import data
runtime_start()
data = sc.read_10x_mtx('./data/filtered_gene_bc_matrices/hg19/', var_names='gene_symbols', cache=True)

# Filter useless data
sc.pp.filter_genes(data, min_cells=1)
filtered_data = np.array(data._X.todense())

# PCA
pca = PCA(n_components=2)
pca_data = pca.fit_transform(filtered_data)
print(sum(pca.explained_variance_ratio_))
print(pca.singular_values_)

def random_patients():
    # Create Centroid Array by randomly picking 5 patients from data  
    k = 5
    patients = pca_data.shape[0]
    genes = pca_data.shape[1]
    centroids_numbers = np.random.randint(patients, size=k)
    centroids_array = np.empty([0, genes])
    i = 0

def random_start_centroids(k):
    # Pick random start sample 
    while i < k:
        randompatient = centroids_numbers[i]
        centroids_array = np.append(centroids_array, [pca_data[randompatient, :]], axis = 0)
        i += 1

def assign_centroids():
    # Assign closest Centroid
    # Loop über alle Punkte
    i = 0
    nearest_centroid = np.zeros([patients, 1])    
    while (i < patients):
        sml_distance = 0

        # While loop selecting every centroid
        j = 0
        while (j < k):

            if sml_distance == 0 or dist(i,j) < sml_distance:
                sml_distance = dist(i,j)
                nearest_centroid[i, 0] = j
            j += 1
        i += 1

def kmeans():
    random_patients()
    random_start_centroids()
    assign_centroids()

kmeans()

runtime_end()

0.4799913316965103
[3608.9749 3205.3806]


TypeError: random_start_centroids() missing 1 required positional argument: 'k'

In [25]:
# Import libraries
import numpy as np
import pandas
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import scanpy as sc
from datetime import datetime
from sklearn.decomposition import PCA

# Global Variables
t1 = 0
patients = 0
genes = 0
k = 0

# Functions

# Define distance function which takes integer inputs which identify patient and centroid
def dist(patient_point, cluster_number):
    a = pca_data[patient_point, :]
    b = centroids_array[cluster_number, :]
    dist = np.linalg.norm(a-b)
    return dist

def runtime_start():
    global t1 
    t1 = datetime.now().time()

def runtime_end():
    t2 = datetime.now().time()
    FMT = '%H:%M:%S.%f'
    elapsed = str(datetime.strptime(str(t2), FMT) - datetime.strptime(str(t1), FMT))
    return elapsed

# Import data
runtime_start()
data = sc.read_10x_mtx('./data/filtered_gene_bc_matrices/hg19/', var_names='gene_symbols', cache=True)

# Filter useless data
sc.pp.filter_genes(data, min_cells=1)
filtered_data = np.array(data._X.todense())

# PCA
pca = PCA(n_components=2)
pca_data = pca.fit_transform(filtered_data)
print(sum(pca.explained_variance_ratio_))
print(pca.singular_values_)

def random_patients():
    # Create Centroid Array by randomly picking 5 patients from data  
    k = 5
    patients = pca_data.shape[0]
    genes = pca_data.shape[1]
    centroids_numbers = np.random.randint(patients, size=k)
    centroids_array = np.empty([0, genes])
    i = 0

def random_start_centroids(k):
    # Pick random start sample 
    while i < k:
        randompatient = centroids_numbers[i]
        centroids_array = np.append(centroids_array, [pca_data[randompatient, :]], axis = 0)
        i += 1

def assign_centroids():
    # Assign closest Centroid
    # Loop über alle Punkte
    i = 0
    nearest_centroid = np.zeros([patients, 1])    
    while (i < patients):
        sml_distance = 0

        # While loop selecting every centroid
        j = 0
        while (j < k):

            if sml_distance == 0 or dist(i,j) < sml_distance:
                sml_distance = dist(i,j)
                nearest_centroid[i, 0] = j
            j += 1
        i += 1

def kmeans():
    random_patients()
    random_start_centroids(5)
    assign_centroids()

kmeans()

runtime_end()

0.4799910932779312
[3608.9746 3205.3792]


UnboundLocalError: local variable 'i' referenced before assignment

In [26]:
# Import libraries
import numpy as np
import pandas
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import scanpy as sc
from datetime import datetime
from sklearn.decomposition import PCA

# Global Variables
t1 = 0
patients = 0
genes = 0
k = 0

# Functions

# Define distance function which takes integer inputs which identify patient and centroid
def dist(patient_point, cluster_number):
    a = pca_data[patient_point, :]
    b = centroids_array[cluster_number, :]
    dist = np.linalg.norm(a-b)
    return dist

def runtime_start():
    global t1 
    t1 = datetime.now().time()

def runtime_end():
    t2 = datetime.now().time()
    FMT = '%H:%M:%S.%f'
    elapsed = str(datetime.strptime(str(t2), FMT) - datetime.strptime(str(t1), FMT))
    return elapsed

# Import data
runtime_start()
data = sc.read_10x_mtx('./data/filtered_gene_bc_matrices/hg19/', var_names='gene_symbols', cache=True)

# Filter useless data
sc.pp.filter_genes(data, min_cells=1)
filtered_data = np.array(data._X.todense())

# PCA
pca = PCA(n_components=2)
pca_data = pca.fit_transform(filtered_data)
print(sum(pca.explained_variance_ratio_))
print(pca.singular_values_)

def random_patients():
    # Create Centroid Array by randomly picking 5 patients from data  
    k = 5
    patients = pca_data.shape[0]
    genes = pca_data.shape[1]
    centroids_numbers = np.random.randint(patients, size=k)
    centroids_array = np.empty([0, genes])
    i = 0

def random_start_centroids(k):
    k = k
    # Pick random start sample 
    while i < k:
        randompatient = centroids_numbers[i]
        centroids_array = np.append(centroids_array, [pca_data[randompatient, :]], axis = 0)
        i += 1

def assign_centroids():
    # Assign closest Centroid
    # Loop über alle Punkte
    i = 0
    nearest_centroid = np.zeros([patients, 1])    
    while (i < patients):
        sml_distance = 0

        # While loop selecting every centroid
        j = 0
        while (j < k):

            if sml_distance == 0 or dist(i,j) < sml_distance:
                sml_distance = dist(i,j)
                nearest_centroid[i, 0] = j
            j += 1
        i += 1

def kmeans():
    random_patients()
    random_start_centroids(5)
    assign_centroids()

kmeans()

runtime_end()

0.4799911081790924
[3608.9736 3205.3804]


UnboundLocalError: local variable 'i' referenced before assignment

In [27]:
# Import libraries
import numpy as np
import pandas
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import scanpy as sc
from datetime import datetime
from sklearn.decomposition import PCA

# Global Variables
t1 = 0
patients = 0
genes = 0
k = 0

# Functions

# Define distance function which takes integer inputs which identify patient and centroid
def dist(patient_point, cluster_number):
    a = pca_data[patient_point, :]
    b = centroids_array[cluster_number, :]
    dist = np.linalg.norm(a-b)
    return dist

def runtime_start():
    global t1 
    t1 = datetime.now().time()

def runtime_end():
    t2 = datetime.now().time()
    FMT = '%H:%M:%S.%f'
    elapsed = str(datetime.strptime(str(t2), FMT) - datetime.strptime(str(t1), FMT))
    return elapsed

# Import data
runtime_start()
data = sc.read_10x_mtx('./data/filtered_gene_bc_matrices/hg19/', var_names='gene_symbols', cache=True)

# Filter useless data
sc.pp.filter_genes(data, min_cells=1)
filtered_data = np.array(data._X.todense())

# PCA
pca = PCA(n_components=2)
pca_data = pca.fit_transform(filtered_data)
print(sum(pca.explained_variance_ratio_))
print(pca.singular_values_)

def random_patients():
    # Create Centroid Array by randomly picking 5 patients from data  
    k = 5
    patients = pca_data.shape[0]
    genes = pca_data.shape[1]
    centroids_numbers = np.random.randint(patients, size=k)
    centroids_array = np.empty([0, genes])

def random_start_centroids(k):
    k = k
    i = 0
    # Pick random start sample 
    while i < k:
        randompatient = centroids_numbers[i]
        centroids_array = np.append(centroids_array, [pca_data[randompatient, :]], axis = 0)
        i += 1

def assign_centroids():
    # Assign closest Centroid
    # Loop über alle Punkte
    i = 0
    nearest_centroid = np.zeros([patients, 1])    
    while (i < patients):
        sml_distance = 0

        # While loop selecting every centroid
        j = 0
        while (j < k):

            if sml_distance == 0 or dist(i,j) < sml_distance:
                sml_distance = dist(i,j)
                nearest_centroid[i, 0] = j
            j += 1
        i += 1

def kmeans():
    random_patients()
    random_start_centroids(5)
    assign_centroids()

kmeans()

runtime_end()

0.47999124228954315
[3608.975 3205.38 ]


UnboundLocalError: local variable 'centroids_array' referenced before assignment

In [28]:
# Import libraries
import numpy as np
import pandas
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import scanpy as sc
from datetime import datetime
from sklearn.decomposition import PCA

# Global Variables
t1 = 0
patients = 0
genes = 0
k = 0

# Functions

# Define distance function which takes integer inputs which identify patient and centroid
def dist(patient_point, cluster_number):
    a = pca_data[patient_point, :]
    b = centroids_array[cluster_number, :]
    dist = np.linalg.norm(a-b)
    return dist

def runtime_start():
    global t1 
    t1 = datetime.now().time()

def runtime_end():
    t2 = datetime.now().time()
    FMT = '%H:%M:%S.%f'
    elapsed = str(datetime.strptime(str(t2), FMT) - datetime.strptime(str(t1), FMT))
    return elapsed

# Import data
runtime_start()
data = sc.read_10x_mtx('./data/filtered_gene_bc_matrices/hg19/', var_names='gene_symbols', cache=True)

# Filter useless data
sc.pp.filter_genes(data, min_cells=1)
filtered_data = np.array(data._X.todense())

# PCA
pca = PCA(n_components=2)
pca_data = pca.fit_transform(filtered_data)
print(sum(pca.explained_variance_ratio_))
print(pca.singular_values_)

def random_start_centroids(k):
    # Create Centroid Array by randomly picking 5 patients from data  
    k = 5
    patients = pca_data.shape[0]
    genes = pca_data.shape[1]
    centroids_numbers = np.random.randint(patients, size=k)
    centroids_array = np.empty([0, genes])
    k = k
    i = 0
    # Pick random start sample 
    while i < k:
        randompatient = centroids_numbers[i]
        centroids_array = np.append(centroids_array, [pca_data[randompatient, :]], axis = 0)
        i += 1

def assign_centroids():
    # Assign closest Centroid
    # Loop über alle Punkte
    i = 0
    nearest_centroid = np.zeros([patients, 1])    
    while (i < patients):
        sml_distance = 0

        # While loop selecting every centroid
        j = 0
        while (j < k):

            if sml_distance == 0 or dist(i,j) < sml_distance:
                sml_distance = dist(i,j)
                nearest_centroid[i, 0] = j
            j += 1
        i += 1

def kmeans():
    random_start_centroids(5)
    assign_centroids()

kmeans()

runtime_end()

0.4799911677837372
[3608.9749 3205.3794]


'0:00:01.558339'

In [29]:
# Import libraries
import numpy as np
import pandas
import matplotlib.pyplot as pyplot
import matplotlib.cm as cm
import scanpy as sc
from datetime import datetime
from sklearn.decomposition import PCA

# Global Variables
t1 = 0
patients = 0
genes = 0
k = 0

# General Code
# Import data
runtime_start()
data = sc.read_10x_mtx('./data/filtered_gene_bc_matrices/hg19/', var_names='gene_symbols', cache=True)

# Filter useless data
sc.pp.filter_genes(data, min_cells=1)
filtered_data = np.array(data._X.todense())

# PCA
pca = PCA(n_components=2)
pca_data = pca.fit_transform(filtered_data)
print(sum(pca.explained_variance_ratio_))
print(pca.singular_values_)

# Functions
# Define distance function which takes integer inputs which identify patient and centroid
def dist(patient_point, cluster_number):
    a = pca_data[patient_point, :]
    b = centroids_array[cluster_number, :]
    dist = np.linalg.norm(a-b)
    return dist

def runtime_start():
    global t1 
    t1 = datetime.now().time()

def runtime_end():
    t2 = datetime.now().time()
    FMT = '%H:%M:%S.%f'
    elapsed = str(datetime.strptime(str(t2), FMT) - datetime.strptime(str(t1), FMT))
    return elapsed

def random_start_centroids(k):
    # Create Centroid Array by randomly picking 5 patients from data  
    k = 5
    patients = pca_data.shape[0]
    genes = pca_data.shape[1]
    centroids_numbers = np.random.randint(patients, size=k)
    centroids_array = np.empty([0, genes])
    k = k
    i = 0
    # Pick random start sample 
    while i < k:
        randompatient = centroids_numbers[i]
        centroids_array = np.append(centroids_array, [pca_data[randompatient, :]], axis = 0)
        i += 1

def assign_centroids():
    # Assign closest Centroid
    # Loop über alle Punkte
    i = 0
    nearest_centroid = np.zeros([patients, 1])    
    while (i < patients):
        sml_distance = 0

        # While loop selecting every centroid
        j = 0
        while (j < k):

            if sml_distance == 0 or dist(i,j) < sml_distance:
                sml_distance = dist(i,j)
                nearest_centroid[i, 0] = j
            j += 1
        i += 1

def kmeans():
    random_start_centroids(5)
    assign_centroids()

kmeans()

runtime_end()

0.47999128699302673
[3608.9749 3205.3804]


'0:00:01.709944'